In [2]:
import os
import cv2
import numpy as np
from glob import glob

# Weber contrast function
def weber_contrast(image):
    grayscale = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    I_max = np.max(grayscale)
    I_min = np.min(grayscale)
    contrast = (I_max - I_min) / I_min if I_min > 0 else (I_max - I_min) / 1  # Avoid division by zero
    return contrast

# Blending function: applies pixelation-like blending in blocks
def blend_image(image, block_size, occlusion_percent):
    h, w = image.shape[:2]
    num_blocks_x = w // block_size
    num_blocks_y = h // block_size
    total_blocks = num_blocks_x * num_blocks_y
    
    num_occluded_blocks = int(total_blocks * (occlusion_percent / 100))
    
    # Generate random block indices to occlude
    block_indices = np.random.choice(total_blocks, num_occluded_blocks, replace=False)
    
    blended_image = image.copy()
    for index in block_indices:
        row = (index // num_blocks_x) * block_size
        col = (index % num_blocks_x) * block_size
        # Extract the block
        block = image[row:row+block_size, col:col+block_size]
        # Apply blending by averaging the block to create a soft transition
        mean_color = np.mean(block, axis=(0, 1), dtype=int)
        blended_image[row:row+block_size, col:col+block_size] = mean_color  # Blend with the mean color of the block
    
    return blended_image

# Input and output folders
input_folder = os.path.join("data", "stim_EEG")
output_folder = os.path.join("data", "stim_EEG_pixel")
os.makedirs(output_folder, exist_ok=True)

# Block sizes and occlusion percentages
block_sizes = [8, 16, 32]
occlusion_levels = [30, 70]  # 30% and 70% occlusion

# Load images
image_paths = glob(os.path.join(input_folder, "*.jpg"))

for img_path in image_paths:
    img = cv2.imread(img_path)
    img_name = os.path.basename(img_path).split('.')[0]
    
    original_contrast = weber_contrast(img)
    print(f"Original Weber Contrast ({img_name}): {original_contrast:.4f}")
    
    # Randomly assign different block sizes to each occlusion level (30% and 70%) for this image
    occlusion_block_sizes = {occlusion: np.random.choice(block_sizes) for occlusion in occlusion_levels}

    # Loop over occlusion levels
    for occlusion in occlusion_levels:
        block_size = occlusion_block_sizes[occlusion]  # Get randomly selected block size for this occlusion level
        
        blended_img = blend_image(img, block_size, occlusion)
        blended_contrast = weber_contrast(blended_img)
        
        # Save the image with random block size and occlusion level
        save_path = os.path.join(output_folder, f"{img_name}_block{block_size}_occ{occlusion}.jpg")
        cv2.imwrite(save_path, blended_img)
        print(f"Saved blended image: {save_path} | Weber Contrast: {blended_contrast:.4f}")

In [3]:
import pandas as pd 

# Define the folder containing your images

relative_folder = "./stim_eeg"

# Initialize a list to store image details
image_data = []

# Loop through the images in the folder
for idx, filename in enumerate(sorted(os.listdir(output_folder))):
    if filename.endswith((".png", ".jpg", ".jpeg")):  # Adjust extensions if needed
        image_path = os.path.join(relative_folder, filename)
        condition = "exp"
        image_nr = idx + 1
        category_nr = image_nr

        # Append to list
        image_data.append([image_path, condition, image_nr, category_nr])

# Create a DataFrame
df = pd.DataFrame(image_data, columns=["image_path", "condition", "image_nr", "category_nr"])

# Save as CSV
csv_path = os.path.join(output_folder, "image_paths_eeg.csv")
df.to_csv(csv_path, index=False)

print(f"CSV file saved at: {csv_path}")

CSV file saved at: data\stim_EEG_pixel\image_paths_eeg.csv


In [ ]:
import os
import cv2
import numpy as np
from glob import glob

# Weber contrast function
def weber_contrast(image):
    grayscale = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    I_max = np.max(grayscale)
    I_min = np.min(grayscale)
    contrast = (I_max - I_min) / I_min if I_min > 0 else (I_max - I_min) / 1  # Avoid division by zero
    return contrast

# Occlusion function
def occlude_image(image, block_size, occlusion_percent):
    h, w = image.shape[:2]
    num_blocks_x = w // block_size
    num_blocks_y = h // block_size
    total_blocks = num_blocks_x * num_blocks_y
    
    num_occluded_blocks = int(total_blocks * (occlusion_percent / 100))
    
    # Generate random block indices to occlude
    block_indices = np.random.choice(total_blocks, num_occluded_blocks, replace=False)
    
    occluded_image = image.copy()
    for index in block_indices:
        row = (index // num_blocks_x) * block_size
        col = (index % num_blocks_x) * block_size
        occluded_image[row:row+block_size, col:col+block_size] = (0, 0, 0)  # Set to black
    
    return occluded_image

# Input and output folders
input_folder = r"C:\Users\victo\Desktop\MRes\25MResButler\250129pilot\stim_EEG"
output_folder = r"C:\Users\victo\Desktop\MRes\25MResButler\250129pilot\stim_EEG3"
os.makedirs(output_folder, exist_ok=True)

# Block sizes and occlusion percentages
block_sizes = [8, 16, 32]
occlusion_levels = [30, 70]  # 30% and 70% occlusion

# Load images
image_paths = glob(os.path.join(input_folder, "*.jpg"))

for img_path in image_paths:
    img = cv2.imread(img_path)
    img_name = os.path.basename(img_path).split('.')[0]
    
    original_contrast = weber_contrast(img)
    print(f"Original Weber Contrast ({img_name}): {original_contrast:.4f}")
    
    # Randomly assign different block sizes to each occlusion level (30% and 70%) for this image
    occlusion_block_sizes = {occlusion: np.random.choice(block_sizes) for occlusion in occlusion_levels}

    # Loop over occlusion levels
    for occlusion in occlusion_levels:
        block_size = occlusion_block_sizes[occlusion]  # Get randomly selected block size for this occlusion level
        
        occluded_img = occlude_image(img, block_size, occlusion)
        occluded_contrast = weber_contrast(occluded_img)
        
        # Save the image with random block size and occlusion level
        save_path = os.path.join(output_folder, f"{img_name}_block{block_size}_occ{occlusion}.jpg")
        cv2.imwrite(save_path, occluded_img)
        print(f"Saved occluded image: {save_path} | Weber Contrast: {occluded_contrast:.4f}")


Delete

In [ ]:
import os
from PIL import Image
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage.measure import entropy
from skimage.morphology import disk
import cv2

def check_pixelation_level(image_path):
    """Detects the block size of a pixelated image by comparing sharpness at different scales."""
    img = Image.open(image_path)
    width, height = img.size
    print(f"Image Size: {width}x{height}")

def analyse_frequency(image_path):
    """Performs FFT to analyze the frequency content of an image."""
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    f = np.fft.fft2(img)
    fshift = np.fft.fftshift(f)
    magnitude_spectrum = 20 * np.log(np.abs(fshift))
    
    plt.figure(figsize=(6,6))
    plt.imshow(magnitude_spectrum, cmap='gray')
    plt.title("Frequency Spectrum")
    plt.show()

def detect_edges(image_path):
    """Applies Canny edge detection to observe sharpness levels."""
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    edges = cv2.Canny(img, 50, 150)
    
    plt.figure(figsize=(6,6))
    plt.imshow(edges, cmap='gray')
    plt.title("Edge Detection")
    plt.show()

def calculate_entropy(image_path):
    """Computes entropy to estimate pixelation level."""
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    ent = entropy(img, disk(5))
    
    avg_entropy = ent.mean()
    print(f"Entropy: {avg_entropy:.4f}")
    return avg_entropy

In [ ]:
check_pixelation_level(r"C:\Users\victo\Desktop\MRes\25MResButler\250129pilot\stim_EEG\34.jpg")
detect_edges(r"C:\Users\victo\Desktop\MRes\25MResButler\250129pilot\stim_EEG\34.jpg")
analyse_frequency(r"C:\Users\victo\Desktop\MRes\25MResButler\250129pilot\stim_EEG\34.jpg")

In [ ]:
import os
import cv2
import numpy as np
from glob import glob

def pixelate_image(image, block_size):
    """
    Pixelates an image by resizing it down and then up using the given block size.
    """
    h, w = image.shape[:2]
    small = cv2.resize(image, (w // block_size, h // block_size), interpolation=cv2.INTER_LINEAR)
    pixelated = cv2.resize(small, (w, h), interpolation=cv2.INTER_NEAREST)
    return pixelated

input_folder = r"C:\Users\victo\Desktop\MRes\25MResButler\250129pilot\stim_EEG"
output_folder = r"C:\Users\victo\Desktop\MRes\25MResButler\250129pilot\stim_EEG2"
os.makedirs(output_folder, exist_ok=True)

# Pixelation levels
block_sizes = [8, 16, 32]  # Mild, moderate, strong pixelation

# Load images
image_paths = glob(os.path.join(input_folder, "*.jpg"))  

for img_path in image_paths:
    img = cv2.imread(img_path)
    img_name = os.path.basename(img_path).split('.')[0]
    
    for block_size in block_sizes:
        pixelated = pixelate_image(img, block_size)
        save_path = os.path.join(output_folder, f"{img_name}_pixel_{block_size}.jpg")
        cv2.imwrite(save_path, pixelated)
        print(f"Saved pixelated image: {save_path}")

In [ ]:
import pandas as pd 

# Define the folder containing your images
image_folder = r"C:\Users\victo\Desktop\MRes\25MResButler\250129pilot\stim_EEG2"  # Update this to your actual folder path
relative_folder = "./stim_eeg"

# Initialize a list to store image details
image_data = []

# Loop through the images in the folder
for idx, filename in enumerate(sorted(os.listdir(image_folder))):
    if filename.endswith((".png", ".jpg", ".jpeg")):  # Adjust extensions if needed
        image_path = os.path.join(relative_folder, filename)
        condition = "exp"
        image_nr = idx + 1
        category_nr = image_nr

        # Append to list
        image_data.append([image_path, condition, image_nr, category_nr])

# Create a DataFrame
df = pd.DataFrame(image_data, columns=["image_path", "condition", "image_nr", "category_nr"])

# Save as CSV
csv_path = os.path.join(image_folder, "image_paths_eeg.csv")
df.to_csv(csv_path, index=False)

print(f"CSV file saved at: {csv_path}")